In [88]:
from abc import ABC, abstractmethod
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C as sb_A2C
from stable_baselines3 import PPO as sb_PPO
from stable_baselines3 import DDPG as sb_DDPG
from stable_baselines3 import TD3 as sb_TD3
from stable_baselines3.common.noise import ActionNoise
from stable_baselines3.common.buffers import ReplayBuffer
from stable_baselines3.common.type_aliases import Schedule
from gym import spaces
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch as th
import yaml
import gym
from datetime import datetime
from sys import path
from os.path import dirname as dir
from typing import Any, Dict, Optional, Tuple, Type, Union
path.append(dir(path[0]))
print(path)
#__package__ = "examples"

from FinancialDataLayer.DataCollection.DataDownloader import DataDownloader
from FinancialDataLayer.DataProcessing.DefaultFeatureEngineer import DefaultFeatureEngineer
from AgentLayer.DataSplitter.TimeSeriesSplitter import TimeSeriesSplitter
from AgentLayer.DataSplitter.BlockingTimeSeriesSplitter import BlockingTimeSeriesSplitter

['/Users/doganparlak/Desktop/Master_2.2/Master_Project/uniFi_github/uniFi/AgentLayer', '/usr/local/Cellar/python@3.8/3.8.8_1/Frameworks/Python.framework/Versions/3.8/lib/python38.zip', '/usr/local/Cellar/python@3.8/3.8.8_1/Frameworks/Python.framework/Versions/3.8/lib/python3.8', '/usr/local/Cellar/python@3.8/3.8.8_1/Frameworks/Python.framework/Versions/3.8/lib/python3.8/lib-dynload', '', '/Users/doganparlak/Library/Python/3.8/lib/python/site-packages', '/usr/local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/site-packages/selenium-3.141.0-py3.8.egg', '/usr/local/lib/python3.8/site-packages/urllib3-1.26.4-py3.8.egg', '/Users/doganparlak/Desktop/Master_2.2/Master_Project/uniFi_github/uniFi', '/Users/doganparlak/Desktop/Master_2.2/Master_Project/uniFi_github/uniFi', '/Users/doganparlak/Desktop/Master_2.2/Master_Project/uniFi_github/uniFi', '/Users/doganparlak/Desktop/Master_2.2/Master_Project/uniFi_github/uniFi', '/Users/doganparlak/Desktop/Master_2.2/Master_Project/uniFi_githu

In [89]:
import pickle
import pypfopt
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import objective_functions
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [4]:
class Environment(gym.Env, ABC):

    @abstractmethod
    def reset(self):
        pass

    @abstractmethod
    def step(self, action):
        pass

    @abstractmethod
    def render(self, mode="human"):
        pass

    @abstractmethod
    def get_env(self):
        pass

    @staticmethod
    def softmax_normalization(actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator / denominator
        return softmax_output


In [67]:
class Agent(ABC):

    @abstractmethod
    def train_model():
        pass

    @abstractmethod
    def predict():
        pass

    @abstractmethod
    def save_model():
        pass
    
    @abstractmethod
    def load_model():
        pass

In [68]:
class ConventionalAgent(Agent, ABC):

    @abstractmethod
    def train_model():
        pass

    @abstractmethod
    def predict():
        pass

    @abstractmethod
    def save_model():
        pass

    @abstractmethod
    def load_model():
        pass
    
    @abstractmethod
    def _return_predict():
        pass

    @abstractmethod
    def _weight_optimization():
        pass

In [8]:
class RLAgent(Agent, ABC):

    @abstractmethod
    def train_model():
        pass

    @abstractmethod
    def predict():
        
        pass

    @abstractmethod
    def save_model():
        pass

    @abstractmethod
    def load_model():
        pass

    @staticmethod
    def extract_weights(rl_actions_list):
        agent_weight_df = {'date': [], 'weights': []}
        for i in range(len(rl_actions_list)):
            date = rl_actions_list.index[i]
            tic_list = list(rl_actions_list.columns)
            weights_list = rl_actions_list.reset_index()[list(rl_actions_list.columns)].iloc[i].values
            weight_dict = {'tic': [], 'weight': []}
            for j in range(len(tic_list)):
                weight_dict['tic'] += [tic_list[j]]
                weight_dict['weight'] += [weights_list[j]]

            agent_weight_df['date'] += [date]
            agent_weight_df['weights'] += [pd.DataFrame(weight_dict)]

        agent_weights = pd.DataFrame(agent_weight_df)
        return agent_weights

In [59]:
class RFAgent(ConventionalAgent):

    def __init__(self,
                n_estimators = 100,
                criterion = "squared_error",
                max_depth = None,
                min_samples_split = 2,
                min_samples_leaf = 1,
                min_weight_fraction_leaf = 0,
                max_features = 1,
                max_leaf_nodes = None,
                min_impurity_decrease = 0,
                bootstrap = True,
                oob_score = False,
                n_jobs = None,
                random_state = None,
                verbose = 0,
                warm_start = False,
                ccp_alpha = 0,
                max_samples = None):

        self.model = RandomForestRegressor(n_estimators= n_estimators,
                            criterion= criterion,
                            max_depth=max_depth,
                            min_samples_split=min_samples_split,
                            min_samples_leaf=min_samples_leaf,
                            min_weight_fraction_leaf=min_weight_fraction_leaf,
                            max_features=max_features,
                            max_leaf_nodes= max_leaf_nodes,
                            min_impurity_decrease=min_impurity_decrease,
                            bootstrap=bootstrap,
                            oob_score=oob_score,
                            n_jobs=n_jobs,
                            random_state= random_state,
                            verbose=verbose,
                            warm_start=warm_start,
                            ccp_alpha=ccp_alpha,
                            max_samples=max_samples)
    
    def train_model(self, train_x, train_y, **train_params):
        '''
        *Trains the model*
        Input: Train data x and train data y
        Output: Linear Regression Model
        '''
        try:
            trained = self.model.fit(train_x, train_y.ravel(), **train_params)
            print("Model trained succesfully")
            return trained
        except Exception as e:
            print("training unsuccessful")    
    
    def predict(self,
                test_data, 
                initial_capital = 0,
                tech_indicator_list = [
                        "macd",
                        "boll_ub",
                        "boll_lb",
                        "rsi_30",
                        "cci_30",
                        "dx_30",
                        "close_30_sma",
                        "close_60_sma",
                    ]):

            meta_coefficient = {"date": [], "weights": []}
            unique_trade_date = test_data.date.unique()
            portfolio = pd.DataFrame(index=range(1), columns=unique_trade_date)
            portfolio.loc[0, unique_trade_date[0]] = initial_capital

            for i in range(len(unique_trade_date) - 1):
                mu, sigma, tics, df_current, df_next = self._return_predict(
                    unique_trade_date, test_data, i, tech_indicator_list)

                portfolio_value = self._weight_optimization(
                    i, unique_trade_date, meta_coefficient, mu, sigma, tics, portfolio, df_current, df_next)
        
            portfolio = portfolio_value
            portfolio = portfolio.T
            portfolio.columns = ['account_value']
            portfolio = portfolio.reset_index()
            portfolio.columns = ['date', 'account_value']
            
            return portfolio, meta_coefficient
    
    def _return_predict(self, unique_trade_date, test_data, i, tech_indicator_list):

        current_date = unique_trade_date[i]
        next_date = unique_trade_date[i+1]

        df_current = test_data[test_data.date ==
                                  current_date].reset_index(drop=True)
        df_next = test_data[test_data.date ==
                               next_date].reset_index(drop=True)

        tics = df_current['tic'].values
        features = df_current[tech_indicator_list].values

        predicted_y = self.model.predict(features)
        mu = predicted_y
        sigma = risk_models.sample_cov(
            df_current.return_list[0], returns_data=True)

        return mu, sigma, tics, df_current, df_next
    
    def _weight_optimization(self, i, unique_trade_date, meta_coefficient, mu, sigma, tics, portfolio, df_current, df_next):

        current_date = unique_trade_date[i]
        predicted_y_df = pd.DataFrame(
            {"tic": tics.reshape(-1,), "predicted_y": mu.reshape(-1,)})
        min_weight, max_weight = 0, 1

        ef = EfficientFrontier(mu, sigma)
        weights = ef.nonconvex_objective(
            objective_functions.sharpe_ratio,
            objective_args=(ef.expected_returns, ef.cov_matrix),
            weights_sum_to_one=True,
            constraints=[
                # greater than min_weight
                {"type": "ineq", "fun": lambda w: w - min_weight},
                # less than max_weight
                {"type": "ineq", "fun": lambda w: max_weight - w},
            ],
        )

        weight_df = {"tic": [], "weight": []}
        meta_coefficient["date"] += [current_date]

        for item in weights:
            weight_df['tic'] += [item]
            weight_df['weight'] += [weights[item]]

        weight_df = pd.DataFrame(weight_df).merge(predicted_y_df, on=['tic'])
        meta_coefficient["weights"] += [weight_df]
        cap = portfolio.iloc[0, i]
        # current cash invested for each stock
        current_cash = [element * cap for element in list(weights.values())]
        # current held shares
        current_shares = list(np.array(current_cash) / np.array(df_current.close))
        # next time period price
        next_price = np.array(df_next.close)
        portfolio.iloc[0, i+1] = np.dot(current_shares, next_price)

        return portfolio 

    def save_model(self,  file_name):
        with open(file_name, 'wb') as files:
            pickle.dump(self.model, files)
        print("Model saved succesfully.")


    def load_model(self, file_name):
        with open(file_name, 'rb') as f:
            self.model = pickle.load(f)
        print("Model loaded succesfully.")
        return self.model

In [60]:
class DTAgent(ConventionalAgent):
    
    def __init__(self,
                criterion = "squared_error",
                splitter = "best",
                max_depth = None,
                min_samples_split = 2,
                min_samples_leaf = 1,
                min_weight_fraction_leaf = 0,
                max_features = None,
                random_state = None,
                max_leaf_nodes = None,
                min_impurity_decrease = 0,
                ccp_alpha = 0):
                

        self.model = DecisionTreeRegressor( criterion = criterion,
                                            splitter = splitter,
                                            max_depth = max_depth,
                                            min_samples_split = min_samples_split,
                                            min_samples_leaf = min_samples_leaf,
                                            min_weight_fraction_leaf = min_weight_fraction_leaf,
                                            max_features = max_features,
                                            random_state = random_state,
                                            max_leaf_nodes = max_leaf_nodes,
                                            min_impurity_decrease = min_impurity_decrease,
                                            ccp_alpha = ccp_alpha)
                                            
    def train_model(self, train_x, train_y, **train_params):
        '''
        *Trains the model*
        Input: Train data x and train data y
        Output: Linear Regression Model
        '''
        try:
            trained = self.model.fit(train_x, train_y, **train_params)
            print("Model trained succesfully")
            return trained
        except Exception as e:
            print("training unsuccessful")    
    
    def predict(self,
                test_data, 
                initial_capital = 0,
                tech_indicator_list = [
                        "macd",
                        "boll_ub",
                        "boll_lb",
                        "rsi_30",
                        "cci_30",
                        "dx_30",
                        "close_30_sma",
                        "close_60_sma",
                    ]):

            meta_coefficient = {"date": [], "weights": []}
            unique_trade_date = test_data.date.unique()
            portfolio = pd.DataFrame(index=range(1), columns=unique_trade_date)
            portfolio.loc[0, unique_trade_date[0]] = initial_capital

            for i in range(len(unique_trade_date) - 1):
                mu, sigma, tics, df_current, df_next = self._return_predict(
                    unique_trade_date, test_data, i, tech_indicator_list)

                portfolio_value = self._weight_optimization(
                    i, unique_trade_date, meta_coefficient, mu, sigma, tics, portfolio, df_current, df_next)
        
            portfolio = portfolio_value
            portfolio = portfolio.T
            portfolio.columns = ['account_value']
            portfolio = portfolio.reset_index()
            portfolio.columns = ['date', 'account_value']

            return portfolio, meta_coefficient
            
    def _return_predict(self, unique_trade_date, test_data, i, tech_indicator_list):

        current_date = unique_trade_date[i]
        next_date = unique_trade_date[i+1]

        df_current = test_data[test_data.date ==
                                  current_date].reset_index(drop=True)
        df_next = test_data[test_data.date ==
                               next_date].reset_index(drop=True)

        tics = df_current['tic'].values
        features = df_current[tech_indicator_list].values

        predicted_y = self.model.predict(features)
        mu = predicted_y
        sigma = risk_models.sample_cov(
            df_current.return_list[0], returns_data=True)

        return mu, sigma, tics, df_current, df_next

    def _weight_optimization(self, i, unique_trade_date, meta_coefficient, mu, sigma, tics, portfolio, df_current, df_next):

        current_date = unique_trade_date[i]
        predicted_y_df = pd.DataFrame(
            {"tic": tics.reshape(-1,), "predicted_y": mu.reshape(-1,)})
        min_weight, max_weight = 0, 1

        ef = EfficientFrontier(mu, sigma)
        weights = ef.nonconvex_objective(
            objective_functions.sharpe_ratio,
            objective_args=(ef.expected_returns, ef.cov_matrix),
            weights_sum_to_one=True,
            constraints=[
                # greater than min_weight
                {"type": "ineq", "fun": lambda w: w - min_weight},
                # less than max_weight
                {"type": "ineq", "fun": lambda w: max_weight - w},
            ],
        )

        weight_df = {"tic": [], "weight": []}
        meta_coefficient["date"] += [current_date]

        for item in weights:
            weight_df['tic'] += [item]
            weight_df['weight'] += [weights[item]]

        weight_df = pd.DataFrame(weight_df).merge(predicted_y_df, on=['tic'])
        meta_coefficient["weights"] += [weight_df]
        cap = portfolio.iloc[0, i]
        # current cash invested for each stock
        current_cash = [element * cap for element in list(weights.values())]
        # current held shares
        current_shares = list(np.array(current_cash) / np.array(df_current.close))
        # next time period price
        next_price = np.array(df_next.close)
        portfolio.iloc[0, i+1] = np.dot(current_shares, next_price)

        return portfolio 
    
    def save_model(self,  file_name):
        with open(file_name, 'wb') as files:
            pickle.dump(self.model, files)
        print("Model saved succesfully.")


    def load_model(self, file_name):
        with open(file_name, 'rb') as f:
            self.model = pickle.load(f)
        print("Model loaded succesfully.")
        return self.model

In [158]:
x = np.asarray([0.25] * 5)
np.reshape(x,(len(x),1))

array([[0.25],
       [0.25],
       [0.25],
       [0.25],
       [0.25]])

In [208]:
class SVRAgent(ConventionalAgent):
    
    def __init__(self,
                kernel = 'rbf',
                degree = 3,
                gamma = 'scale',
                coef0 = 0,
                tol = 0.001,
                C = 1,
                epsilon = 0.1,
                shrinking = True,
                cache_size = 200,
                verbose = False,
                max_iter = -1):  

        self.model = SVR(kernel= kernel,
                        degree= degree,
                        gamma= gamma, 
                        coef0= coef0,
                        tol= tol,
                        C= C,
                        epsilon= epsilon,
                        shrinking= shrinking,
                        cache_size= cache_size,
                        verbose= verbose,
                        max_iter= max_iter)
            
    def get_params(self, deep = True):
        return self.model.get_params(deep = deep)

    def train_model(self, train_x, train_y, **train_params):
        '''
        *Trains the model*
        Input: Train data x and train data y
        Output: Linear Regression Model
        '''
        try:
            trained = self.model.fit(train_x, train_y.ravel(), **train_params)
            print("Model trained succesfully")
            return trained
        except Exception as e:
            print("training unsuccessful")

    def predict(self,
               test_data, 
               initial_capital = 1000000,
               transaction_cost_pct = 0.001,
               tech_indicator_list = [
                    "macd",
                    "boll_ub",
                    "boll_lb",
                    "rsi_30",
                    "cci_30",
                    "dx_30",
                    "close_30_sma",
                    "close_60_sma",
                ]):

        meta_coefficient = {"date": []}
        for i in test_data.tic:
            meta_coefficient[i] = []
        unique_trade_date = test_data.date.unique()
        weight_arr = [np.array([1/len(test_data.tic.unique())]*len(test_data.tic.unique()))]
        portfolio = pd.DataFrame(index=range(1), columns=unique_trade_date)
        portfolio.loc[0, unique_trade_date[0]] = initial_capital
        for i in range(len(unique_trade_date) - 1):
            mu, sigma, tics, df_current, df_next = self._return_predict(
                unique_trade_date, test_data, i, tech_indicator_list)

            portfolio_value, weight_arr = self._weight_optimization(
                i, unique_trade_date, meta_coefficient, mu, sigma, tics, portfolio, df_current, df_next, transaction_cost_pct, weight_arr)
    
        portfolio = portfolio_value
        portfolio = portfolio.T
        portfolio.columns = ['account_value']
        portfolio = portfolio.reset_index()
        portfolio.columns = ['date', 'account_value']
        
        meta_coefficient = pd.DataFrame(meta_coefficient).set_index("date")
        return portfolio, meta_coefficient
    
    def _return_predict(self, unique_trade_date, test_data, i, tech_indicator_list):

        current_date = unique_trade_date[i]
        next_date = unique_trade_date[i+1]

        df_current = test_data[test_data.date ==
                                  current_date].reset_index(drop=True)
        df_next = test_data[test_data.date ==
                               next_date].reset_index(drop=True)

        tics = df_current['tic'].values
        features = df_current[tech_indicator_list].values

        predicted_y = self.model.predict(features)
        mu = predicted_y
        sigma = risk_models.sample_cov(
            df_current.return_list[0], returns_data=True)

        return mu, sigma, tics, df_current, df_next
    
    def _weight_optimization(self, i, unique_trade_date, meta_coefficient, mu, sigma, tics, portfolio, df_current, df_next, transaction_cost_pct, weight_arr):
        current_date = unique_trade_date[i]
        predicted_y_df = pd.DataFrame(
            {"tic": tics.reshape(-1,), "predicted_y": mu.reshape(-1,)})
        min_weight, max_weight = 0, 1

        ef = EfficientFrontier(mu, sigma)
        w_prev = np.array(weight_arr[-1],dtype=object)
        print(w_prev)
        print("---------------")
        ef.add_objective(objective_functions.transaction_cost, w_prev = w_prev, k = transaction_cost_pct)
        weights = ef.nonconvex_objective(
            objective_functions.sharpe_ratio,
            objective_args=(ef.expected_returns, ef.cov_matrix),
            weights_sum_to_one=True,
            constraints=[
                # greater than min_weight
                {"type": "ineq", "fun": lambda w: w - min_weight},
                # less than max_weight
                {"type": "ineq", "fun": lambda w: max_weight - w},
            ],
        )
   
        
        weight_df = {"tic": [], "weight": []}
        meta_coefficient["date"] += [current_date]

        for item in weights:
            weight_df['tic'] += [item]
            weight_df['weight'] += [weights[item]]

        weight_df = pd.DataFrame(weight_df).merge(predicted_y_df, on=['tic'])

        tics_list = list(weight_df['tic'])
        weights_list = list(weight_df['weight'])
        new_weights = []
        for j in range(len(tics_list)):
            meta_coefficient[tics_list[j]] += [weights_list[j]]
            new_weights.append(weights_list[j])
        weight_arr.append(new_weights)
        cap = portfolio.iloc[0, i]
        # current cash invested for each stock
        current_cash = [element * cap for element in list(weights.values())]
        # current held shares
        current_shares = list(np.array(current_cash) / np.array(df_current.close))
        # next time period price
        next_price = np.array(df_next.close)
        portfolio.iloc[0, i+1] = np.dot(current_shares, next_price)

        return portfolio , weight_arr

    
    def save_model(self,  file_name):
        with open(file_name, 'wb') as files:
            pickle.dump(self.model, files)
        print("Model saved succesfully.")


    def load_model(self, file_name):
        with open(file_name, 'rb') as f:
            self.model = pickle.load(f)
        print("Model loaded succesfully.")
        return self.model

In [102]:
class LRAgent(ConventionalAgent):

    def __init__(self,
                 fit_intercept=True,
                 copy_X=True,
                 positive=False):
        """

        @param fit_intercept:
        @param copy_X:
        @param positive:
        """
        print(fit_intercept)
        self.model = LinearRegression(fit_intercept=fit_intercept,
                                      copy_X=copy_X,
                                      positive=positive)

    def train_model(self, train_x, train_y, **train_params):
        """

        @param train_x:
        @param train_y:
        @param train_params:
        @return:
        """
        try:
            trained_reg = self.model.fit(train_x, train_y, **train_params)
            self.model = trained_reg
            print("Model trained succesfully")
        except Exception as e:
            print("training unsuccessful")

    def predict(self,
                test_data,
                initial_capital=1000000,
                tech_indicator_list=[
                    "macd",
                    "boll_ub",
                    "boll_lb",
                    "rsi_30",
                    "cci_30",
                    "dx_30",
                    "close_30_sma",
                    "close_60_sma",
                ]
                ):
        """

        @param test_data:
        @param initial_capital:
        @param tech_indicator_list:
        @return:
        """

        meta_coefficient = {"date": []}
        for i in test_data.tic:
            meta_coefficient[i] = []
        unique_trade_date = test_data.date.unique()
        portfolio = pd.DataFrame(index=range(1), columns=unique_trade_date)
        portfolio.loc[0, unique_trade_date[0]] = initial_capital
        for i in range(len(unique_trade_date) - 1):
            mu, sigma, tics, df_current, df_next = self._return_predict(
                unique_trade_date, test_data, i, tech_indicator_list)

            portfolio_value = self._weight_optimization(
                i, unique_trade_date, meta_coefficient, mu, sigma, tics, portfolio, df_current, df_next)
    
        portfolio = portfolio_value
        portfolio = portfolio.T
        portfolio.columns = ['account_value']
        portfolio = portfolio.reset_index()
        portfolio.columns = ['date', 'account_value']
        
        meta_coefficient = pd.DataFrame(meta_coefficient).set_index("date")
        return portfolio, meta_coefficient

    def _return_predict(self, unique_trade_date, test_data, i, tech_indicator_list):
        """

        @param unique_trade_date:
        @param test_data:
        @param i:
        @param tech_indicator_list:
        @return:
        """
        current_date = unique_trade_date[i]
        next_date = unique_trade_date[i + 1]

        df_current = test_data[test_data.date ==
                               current_date].reset_index(drop=True)
        df_next = test_data[test_data.date ==
                            next_date].reset_index(drop=True)

        tics = df_current['tic'].values
        features = df_current[tech_indicator_list].values

        predicted_y = self.model.predict(features)
        mu = predicted_y
        sigma = risk_models.sample_cov(
            df_current.return_list[0], returns_data=True)

        return mu, sigma, tics, df_current, df_next

    def _weight_optimization(self, i, unique_trade_date, meta_coefficient, mu, sigma, tics, portfolio, df_current,
                             df_next):
        """

        @param i:
        @param unique_trade_date:
        @param meta_coefficient:
        @param mu:
        @param sigma:
        @param tics:
        @param portfolio:
        @param df_current:
        @param df_next:
        @return:
        """
        current_date = unique_trade_date[i]
        predicted_y_df = pd.DataFrame(
            {"tic": tics.reshape(-1,), "predicted_y": mu.reshape(-1,)})
        min_weight, max_weight = 0, 1

        ef = EfficientFrontier(mu, sigma)
        weights = ef.nonconvex_objective(
            objective_functions.sharpe_ratio,
            objective_args=(ef.expected_returns, ef.cov_matrix),
            weights_sum_to_one=True,
            constraints=[
                # greater than min_weight
                {"type": "ineq", "fun": lambda w: w - min_weight},
                # less than max_weight
                {"type": "ineq", "fun": lambda w: max_weight - w},
            ],
        )
     
        weight_df = {"tic": [], "weight": []}
        meta_coefficient["date"] += [current_date]

        for item in weights:
            weight_df['tic'] += [item]
            weight_df['weight'] += [weights[item]]

        weight_df = pd.DataFrame(weight_df).merge(predicted_y_df, on=['tic'])

        tics_list = list(weight_df['tic'])
        weights_list = list(weight_df['weight'])
        for j in range(len(tics_list)):
            meta_coefficient[tics_list[j]] += [weights_list[j]]

        cap = portfolio.iloc[0, i]
        # current cash invested for each stock
        current_cash = [element * cap for element in list(weights.values())]
        # current held shares
        current_shares = list(np.array(current_cash) / np.array(df_current.close))
        # next time period price
        next_price = np.array(df_next.close)
        portfolio.iloc[0, i+1] = np.dot(current_shares, next_price)

        return portfolio 

    def save_model(self, file_name):
        """

        @param file_name:
        @return:
        """
        with open(file_name, 'wb') as files:
            pickle.dump(self.model, files)
        print("Model saved succesfully.")

    def load_model(self, file_name):
        """

        @param file_name:
        @return:
        """
        with open(file_name, 'rb') as f:
            self.model = pickle.load(f)
        print("Model loaded succesfully.")
        return self.model


In [7]:
class PortfolioEnv(Environment):

    def __init__(self,
                 df: pd.DataFrame,  # input data
                 stock_dim: int,  # number of unique securities in the investment universe
                 hmax: float,  # maximum number of shares to trade
                 initial_amount: float,  # initial cash value
                 transaction_cost_pct: float,  # transaction cost percentage per trade
                 reward_scaling: float,  # scaling factor for reward as training progresses
                 state_space: int,  # the dimension of input features (state space)
                 action_space: int,  # number of actions, which is equal to portfolio dimension
                 tech_indicator_list: list,  # a list of technical indicator names
                 lookback=252,  #
                 day=0):  # an increment number to control date

        self.df = df
        self.day = day
        self.lookback = lookback
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct = transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low=0, high=1, shape=(self.action_space,))
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf,
                                            shape=(self.state_space + len(self.tech_indicator_list), self.state_space))
        

        # load data from a pandas dataframe
        
        
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False
        #self.turbulence_threshold = turbulence_threshold
        # initalize state: initial portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory = [[1 / self.stock_dim] * self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
                
        self.portfolio_value = self.initial_amount
        self.terminal = False 
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim] * self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]] 
        return self.state

    def step(self, actions):
        self.terminal = self.day >= len(self.df.index.unique()) - 1
        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() != 0:
                sharpe = (252 ** 0.5) * df_daily_return['daily_return'].mean() / \
                         df_daily_return['daily_return'].std()
                print("Sharpe: ", sharpe)
            print("=================================")

            return self.state, self.reward, self.terminal, {}

        else:
            weights = Environment.softmax_normalization(actions)
            self.actions_memory.append(weights)
            transaction_fee = self.transaction_cost_pct * self.asset_memory[-1] * sum([abs(a_i - b_i) for a_i, b_i in zip(self.actions_memory[-1], self.actions_memory[-2])]) #transaction_fee
            last_day_memory = self.data
            # load next state
            self.day += 1
            self.data = self.df.loc[self.day, :]
            self.covs = self.data['cov_list'].values[0]
            self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)         
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values) - 1) * weights)
            # update portfolio value
            new_portfolio_value = self.portfolio_value * (1 + portfolio_return) - transaction_fee #transaction_fee
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data["date"].unique()[0])
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value

        return self.state, self.reward, self.terminal, {}

    def render(self, mode='human'):
        return self.state

    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        df_account_value = pd.DataFrame({'date': date_list, 'daily_return': portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']

        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        # df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def get_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [50]:
class A2C(RLAgent):

    def __init__(self,
                 policy: "MlpPolicy",
                 env: None,
                 learning_rate: float = 7e-4,
                 n_steps: int = 5,
                 gamma: float = 0.99,
                 gae_lambda: float = 1.0,
                 ent_coef: float = 0.0,
                 vf_coef: float = 0.5,
                 max_grad_norm: float = 0.5,
                 rms_prop_eps: float = 1e-5,
                 use_rms_prop: bool = True,
                 use_sde: bool = False,
                 sde_sample_freq: int = -1,
                 normalize_advantage: bool = False,
                 tensorboard_log: Optional[str] = None,
                 create_eval_env: bool = False,
                 policy_kwargs: Optional[Dict[str, Any]] = None,
                 verbose: int = 0,
                 seed: Optional[int] = None,
                 device: Union[th.device, str] = "auto",
                 _init_setup_model: bool = True):

        self.env = env

        self.model = sb_A2C(policy = policy,
                            env=self.env,
                            learning_rate = learning_rate,
                            n_steps = n_steps,
                            gamma = gamma,
                            gae_lambda= gae_lambda,
                            ent_coef = ent_coef,
                            vf_coef = vf_coef,
                            max_grad_norm = max_grad_norm,
                            rms_prop_eps= rms_prop_eps,
                            use_rms_prop= use_rms_prop,
                            use_sde= use_sde,
                            sde_sample_freq= sde_sample_freq,
                            normalize_advantage= normalize_advantage,
                            tensorboard_log=tensorboard_log,  
                            create_eval_env= create_eval_env,
                            policy_kwargs=policy_kwargs,
                            verbose=verbose,
                            seed=seed,
                            device= device,
                            _init_setup_model = _init_setup_model)

    def train_model(self, **train_params):
        self.model = self.model.learn(**train_params)
        return self.model

    def predict(self, environment, **test_params):

        env_test, obs_test = environment.get_env()
        """make a prediction"""
        account_memory = []
        actions_memory = []

        env_test.reset()
        for i in range(len(environment.df.index.unique())):
            action, _states = self.model.predict(obs_test, **test_params)
            obs_test, rewards, dones, info = env_test.step(action)
            if i == (len(environment.df.index.unique()) - 2):
                account_memory = env_test.env_method(method_name="save_asset_memory")
                actions_memory = env_test.env_method(method_name="save_action_memory")
            if dones[0]:
                print("hit end!")
                break
        
        portfolio_df = account_memory[0]
        portfolio_df = portfolio_df.rename(columns={"daily_return": "account_value"})
        portfolio_df.iloc[0, portfolio_df.columns.get_loc("account_value")] = environment.initial_amount
        values = list(portfolio_df["account_value"])
        for i in range(1,len(values)):
            values[i] = (values[i] + 1) * values[i-1]

        portfolio_df["account_value"] = values
        return portfolio_df, actions_memory[0]

    def save_model(self, path):
        self.model.save(path)

    def load_model(self, path):
        self.model = self.model.load(path)
        return self.model

In [56]:
class PPO(RLAgent):
    def __init__(self,
                policy: "MlpPolicy",
                env: None,
                learning_rate:  3e-4,
                n_steps: int = 2048,
                batch_size: int = 64,
                n_epochs: int = 10,
                gamma: float = 0.99,
                gae_lambda: float = 0.95,
                clip_range: Union[float, Schedule] = 0.2,
                clip_range_vf: Union[None, float, Schedule] = None,
                normalize_advantage: bool = True,
                ent_coef: float = 0.0,
                vf_coef: float = 0.5,
                max_grad_norm: float = 0.5,
                use_sde: bool = False,
                sde_sample_freq: int = -1,
                target_kl: Optional[float] = None,
                tensorboard_log: Optional[str] = None,
                create_eval_env: bool = False,
                policy_kwargs: Optional[Dict[str, Any]] = None,
                verbose: int = 0,
                seed: Optional[int] = None,
                device: Union[th.device, str] = "auto",
                _init_setup_model: bool = True):

        self.env = env
        self.model = sb_PPO(policy = policy,
                            env=self.env,
                            learning_rate = learning_rate,
                            n_steps = n_steps,
                            gamma = gamma,
                            batch_size = batch_size,
                            n_epochs = n_epochs,
                            gae_lambda=gae_lambda,
                            clip_range = clip_range,
                            clip_range_vf = clip_range_vf,
                            normalize_advantage=normalize_advantage,
                            ent_coef=ent_coef,
                            vf_coef=vf_coef,
                            max_grad_norm=max_grad_norm,
                            use_sde=use_sde,
                            sde_sample_freq=sde_sample_freq,
                            target_kl=target_kl,
                            tensorboard_log=tensorboard_log,
                            create_eval_env=create_eval_env,
                            policy_kwargs=policy_kwargs,
                            verbose=verbose,
                            seed=seed,
                            device=device,
                            _init_setup_model = _init_setup_model)
    
    def train_model(self, **train_params):
        self.model = self.model.learn(**train_params)
        return self.model
    
    def predict(self, environment, **test_params):

        env_test, obs_test = environment.get_env()
        """make a prediction"""
        account_memory = []
        actions_memory = []

        env_test.reset()
        for i in range(len(environment.df.index.unique())):
            action, _states = self.model.predict(obs_test, **test_params)
            obs_test, rewards, dones, info = env_test.step(action)
            if i == (len(environment.df.index.unique()) - 2):
                account_memory = env_test.env_method(method_name="save_asset_memory")
                actions_memory = env_test.env_method(method_name="save_action_memory")
            if dones[0]:
                print("hit end!")
                break
        
        portfolio_df = account_memory[0]
        portfolio_df = portfolio_df.rename(columns={"daily_return": "account_value"})
        portfolio_df.iloc[0, portfolio_df.columns.get_loc("account_value")] = environment.initial_amount
        values = list(portfolio_df["account_value"])
        for i in range(1,len(values)):
            values[i] = (values[i] + 1) * values[i-1]

        portfolio_df["account_value"] = values
        return portfolio_df, actions_memory[0]
    
    def load_model(self, path):
        self.model = self.model.load(path)
        return self.model
    
    def save_model(self, path):
        self.model.save(path)

In [57]:
class DDPG(RLAgent):
    def __init__(self,
                policy: "MlpPolicy",
                env: None,
                learning_rate : 1e-3,
                buffer_size: 1_000_000,  # 1e6
                learning_starts: 100,
                batch_size:  100,
                tau:  0.005,
                gamma:  0.99,
                train_freq:  1,
                gradient_steps: int = -1,
                action_noise: Optional[ActionNoise] = None,
                replay_buffer_class: Optional[ReplayBuffer] = None,
                replay_buffer_kwargs: Optional[Dict[str, Any]] = None,
                optimize_memory_usage: bool = False,
                tensorboard_log: Optional[str] = None,
                create_eval_env: bool = False,
                policy_kwargs: Optional[Dict[str, Any]] = None,
                verbose: int = 0,
                seed: Optional[int] = None,
                device: Union[th.device, str] = "auto",
                _init_setup_model: bool = True):
                
        self.env = env
    
        self.model = sb_DDPG(policy = policy,
                            env=self.env,
                            learning_rate = learning_rate,
                            buffer_size = buffer_size,
                            learning_starts= learning_starts,
                            batch_size = batch_size,
                            tau = tau,
                            gamma= gamma,
                            train_freq = train_freq,
                            gradient_steps = gradient_steps,
                            action_noise= action_noise,
                            replay_buffer_class= replay_buffer_class,
                            replay_buffer_kwargs= replay_buffer_kwargs,
                            optimize_memory_usage=optimize_memory_usage,
                            tensorboard_log=tensorboard_log,
                            create_eval_env=create_eval_env,
                            policy_kwargs=policy_kwargs,
                            verbose=verbose,
                            seed=seed,
                            device=device,
                            _init_setup_model = _init_setup_model)

    def train_model(self, **train_params):
        self.model = self.model.learn(**train_params)
        return self.model

    def predict(self, environment, **test_params):

            env_test, obs_test = environment.get_env()
            """make a prediction"""
            account_memory = []
            actions_memory = []

            env_test.reset()
            for i in range(len(environment.df.index.unique())):
                action, _states = self.model.predict(obs_test, **test_params)
                obs_test, rewards, dones, info = env_test.step(action)
                if i == (len(environment.df.index.unique()) - 2):
                    account_memory = env_test.env_method(method_name="save_asset_memory")
                    actions_memory = env_test.env_method(method_name="save_action_memory")
                if dones[0]:
                    print("hit end!")
                    break
            
            portfolio_df = account_memory[0]
            portfolio_df = portfolio_df.rename(columns={"daily_return": "account_value"})
            portfolio_df.iloc[0, portfolio_df.columns.get_loc("account_value")] = environment.initial_amount
            values = list(portfolio_df["account_value"])
            for i in range(1,len(values)):
                values[i] = (values[i] + 1) * values[i-1]

            portfolio_df["account_value"] = values
            return portfolio_df, actions_memory[0]
            
    def load_model(self, path):
        self.model = self.model.load(path)
        return self.model
    
    def save_model(self, path):
        self.model.save(path)

In [58]:
class TD3(RLAgent):
 
    def __init__(self,
                policy: "MlpPolicy",
                env: None,
                learning_rate: float =  1e-3,
                buffer_size: int = 1_000_000,  # 1e6
                learning_starts: int = 100,
                batch_size: int = 100,
                tau: float = 0.005,
                gamma: float = 0.99,
                train_freq: int = 1,
                gradient_steps: int = -1,
                action_noise: Optional[ActionNoise] = None,
                replay_buffer_class: Optional[ReplayBuffer] = None,
                replay_buffer_kwargs: Optional[Dict[str, Any]] = None,
                optimize_memory_usage: bool = False,
                tensorboard_log: Optional[str] = None,
                create_eval_env: bool = False,
                policy_kwargs: Optional[Dict[str, Any]] = None,
                verbose: int = 0,
                seed: Optional[int] = None,
                device: Union[th.device, str] = "auto",
                _init_setup_model: bool = True):
                
        self.env = env
    
        self.model = sb_TD3(policy = policy,
                            env=self.env,
                            learning_rate = learning_rate,
                            buffer_size = buffer_size,
                            learning_starts= learning_starts,
                            batch_size = batch_size,
                            tau = tau,
                            gamma= gamma,
                            train_freq = train_freq,
                            gradient_steps = gradient_steps,
                            action_noise= action_noise,
                            replay_buffer_class= replay_buffer_class,
                            replay_buffer_kwargs= replay_buffer_kwargs,
                            optimize_memory_usage=optimize_memory_usage,
                            tensorboard_log=tensorboard_log,
                            create_eval_env=create_eval_env,
                            policy_kwargs=policy_kwargs,
                            verbose=verbose,
                            seed=seed,
                            device=device,
                            _init_setup_model = _init_setup_model)

    def train_model(self, **train_params):
        self.model = self.model.learn(**train_params)
        return self.model

    def predict(self, environment, **test_params):

        env_test, obs_test = environment.get_env()
        """make a prediction"""
        account_memory = []
        actions_memory = []

        env_test.reset()
        for i in range(len(environment.df.index.unique())):
            action, _states = self.model.predict(obs_test, **test_params)
            obs_test, rewards, dones, info = env_test.step(action)
            if i == (len(environment.df.index.unique()) - 2):
                account_memory = env_test.env_method(method_name="save_asset_memory")
                actions_memory = env_test.env_method(method_name="save_action_memory")
            if dones[0]:
                print("hit end!")
                break
        
        portfolio_df = account_memory[0]
        portfolio_df = portfolio_df.rename(columns={"daily_return": "account_value"})
        portfolio_df.iloc[0, portfolio_df.columns.get_loc("account_value")] = environment.initial_amount
        values = list(portfolio_df["account_value"])
        for i in range(1,len(values)):
            values[i] = (values[i] + 1) * values[i-1]

        portfolio_df["account_value"] = values
        return portfolio_df, actions_memory[0]
        
    def load_model(self, path):
        self.model = self.model.load(path)
        return self.model
    
    def save_model(self, path):
        self.model.save(path)

In [91]:
#Gather user parameters
with open("../user_params.yaml", "r") as stream:
    try:
        user_params = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [92]:
tickers = user_params["TICKERS"]
env_kwargs = user_params["ENV_PARAMS"]
train_params = user_params["TRAIN_PARAMS"]
policy_params = user_params["POLICY_PARAMS"]
test_params = user_params["TEST_PARAMS"]

In [93]:
print("\nTest 3: Downloading from Yahoo.........")
downloaded_df = DataDownloader(start_date='2009-01-01',
                                end_date='2021-10-31',
                                ticker_list= tickers).download_from_yahoo()
print(downloaded_df.head())


Test 3: Downloading from Yahoo.........
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (12924, 8)
        date       open       high        low      close     volume   tic  day
0 2008-12-31   3.070357   3.133571   3.047857   2.606277  607541200  AAPL    2
1 2008-12-31  41.590000  43.049999  41.500000  32.005890    5443100    BA    2
2 2008-12-31  43.700001  45.099998  43.700001  30.628826    6277400   CAT    2
3 2008-12-31  72.900002  74.629997  72.900002  43.314426    9964300   CVX    2
4 2009-01-02   3.067143   3.251429   3.041429   2.771173  746015200  AAPL    4


In [83]:
 # PREPROCESS DATA
print("\nTest 4: Feature engineer.........")
data_processor = DefaultFeatureEngineer(use_default=False,
                                        tech_indicator_list=env_kwargs["tech_indicator_list"],
                                        use_vix=True,
                                        use_turbulence=True,
                                        use_covar=True)
# included technical indicators as features
df_processed = data_processor.extend_data(downloaded_df)


Test 4: Feature engineer.........
Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (3231, 8)
Successfully added vix
Successfully added turbulence index
Successfully added covariances
Successfully added returns


In [94]:
splitter = TimeSeriesSplitter()
train = splitter.get_split_data(df_processed, '2009-01-01', '2020-06-30')
trade = splitter.get_split_data(df_processed, '2020-07-01', '2021-09-02')

In [95]:
#prepare conventional data
X, y = data_processor.prepare_ml_data(train)

In [30]:
policy_params["SVR_PARAMS"].pop('C', None)

{'kernel': 'rbf', 'degree': 3, 'gamma': 'scale', 'coef0': 0, 'tol': 0.001, 'epsilon': 0.1, 'shrinking': True, 'cache_size': 200, 'verbose': False, 'max_iter': -1}


In [31]:
svr_params = policy_params["SVR_PARAMS"]

In [17]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

In [ ]:
cv = BlockingTimeSeriesSplitter(n_splits = 5)
for i,j in cv.split(X,y):
    print(i)
    print(j)

In [ ]:
CC= [0.01, 1, 100]
svr_scores = []
for c in CC:
    btss = BlockingTimeSeriesSplitter(n_splits = 5)
    svr = SVRAgent(C =c, **svr_params)
    scores = []
    for train_idx,test_idx in btss.split(X,y):
       svr.train_model(X[train_idx], y[train_idx], **train_params["SVR_PARAMS"])
       temp_portfolio,_,_ = svr.predict(X[test_idx], **test_params["SVR_PARAMS"])
       scores.append(temp_portfolio.tail(1)["account_value"][0])
       
    scores_mean = sum(scores) / len(scores)
    print(scores_mean)
    svr_scores.append(scores_mean)

In [209]:
# create agent
svr = SVRAgent(**policy_params["SVR_PARAMS"])

In [210]:
svr.train_model(X, y, **train_params["SVR_PARAMS"])

Model trained succesfully


SVR(C=1, coef0=0)

In [211]:
#predict 
portfolio, meta_coefficient = svr.predict(trade, **test_params["SVR_PARAMS"])
print(portfolio)
print("--------\n")
print(meta_coefficient)


[0.25 0.25 0.25 0.25]
---------------
[0.0 1.0 0.0 0.0]
---------------
[5.233631138980191e-17 1.0 0.0 6.6547046648835655e-19]
---------------
[0.0 0.9999999999999998 1.6653345369377348e-16 1.0099357455337315e-16]
---------------
[0.0 0.9999999999999999 3.3306690738754696e-16 0.0]
---------------
[1.0020135657823857e-16 1.0 0.0 6.876750473604274e-19]
---------------
[6.568032439433712e-17 0.9999999999999999 8.326672684688674e-17
 5.626902065834138e-17]
---------------
[0.0 1.0 2.7755575615628914e-17 4.112382106040844e-17]
---------------
[5.161294594603768e-17 1.0 0.0 4.029129968539667e-17]
---------------
[0.0 1.0 2.220446049250313e-16 0.0]
---------------
[6.321234834330744e-17 1.0 0.0 0.0]
---------------
[0.0 1.0 0.0 1.2491826339985263e-17]
---------------
[0.0 1.0 0.0 0.0]
---------------
[1.8134957645801395e-17 1.0 0.0 7.154551451430371e-17]
---------------
[0.0 1.0 1.249000902703301e-16 0.0]
---------------
[5.421606887426042e-17 1.0 0.0 0.0]
---------------
[4.133457043706442e-

In [212]:
portfolio

,date,account_value
0,2020-07-01,1000000
1,2020-07-02,1002717.337037
2,2020-07-06,1042091.814716
3,2020-07-07,992014.183768
4,2020-07-08,998669.002421
...,...,...
291,2021-08-26,1285781.489085
292,2021-08-27,1295139.470885
293,2021-08-30,1322295.09542
294,2021-08-31,1315714.418036


In [213]:
meta_coefficient

,AAPL,BA,CAT,CVX
date,,,,
2020-07-01,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
2020-07-02,5.233631e-17,1.000000e+00,0.000000e+00,6.654705e-19
2020-07-06,0.000000e+00,1.000000e+00,1.665335e-16,1.009936e-16
2020-07-07,0.000000e+00,1.000000e+00,3.330669e-16,0.000000e+00
2020-07-08,1.002014e-16,1.000000e+00,0.000000e+00,6.876750e-19
...,...,...,...,...
2021-08-25,9.349357e-01,2.042637e-16,6.506427e-02,0.000000e+00
2021-08-26,7.268034e-01,0.000000e+00,2.731966e-01,4.172010e-16
2021-08-27,7.439171e-01,2.515349e-17,2.560829e-01,1.778092e-17


In [105]:
portfolio, meta_coefficient = lr.predict(trade, **test_params["LR_PARAMS"])

OrderedDict([('AAPL', 0.0), ('BA', 1.0), ('CAT', 0.0), ('CVX', 0.0)])
OrderedDict([('AAPL', 0.0), ('BA', 1.0), ('CAT', 1.1488641758680014e-17), ('CVX', 0.0)])
OrderedDict([('AAPL', 0.0), ('BA', 1.0), ('CAT', 2.3645032162296554e-17), ('CVX', 0.0)])
OrderedDict([('AAPL', 6.245004513516506e-17), ('BA', 1.0), ('CAT', 0.0), ('CVX', 8.326672684688674e-17)])
OrderedDict([('AAPL', 0.0), ('BA', 1.0), ('CAT', 0.0), ('CVX', 0.0)])
OrderedDict([('AAPL', 1.0408340855860843e-16), ('BA', 1.0), ('CAT', 1.1870850139893797e-17), ('CVX', 0.0)])
OrderedDict([('AAPL', 3.651050815844936e-17), ('BA', 0.9999999999999997), ('CAT', 0.0), ('CVX', 2.6268863386608166e-16)])
OrderedDict([('AAPL', 1.9081958235744878e-17), ('BA', 0.9999999999999998), ('CAT', 2.969519390948734e-17), ('CVX', 2.636779683484747e-16)])
OrderedDict([('AAPL', 6.938893903907228e-17), ('BA', 0.9999999999999999), ('CAT', 1.7219868020753275e-16), ('CVX', 0.0)])
OrderedDict([('AAPL', 1.9081957977250937e-17), ('BA', 1.0), ('CAT', 0.0), ('CVX', 0.

ValueError: not enough values to unpack (expected 3, got 2)